In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error, precision_score, accuracy_score
import os
import pickle
from skimage.transform import resize
from skimage.io import imread
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import tensorflow as tf
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump


#model = MLPClassifier(random_state = 1, learning_rate_init=0.003 ,max_iter=10000)

#print(model)

In [4]:
import os
dataDir = os.path.join('..', 'dataset', 'dataset_')

# Verify the paths exist
print(f"Data directory path: {os.path.abspath(dataDir)}")
print(f"Data directory exists: {os.path.isdir(dataDir)}")

Data directory path: c:\Users\makjo\GithubPrograms\cs306_project2\dataset\dataset_
Data directory exists: False


In [5]:
import numpy as np
import pandas as pd
from skimage.transform import resize
from skimage.io import imread
import os
flat_data_arr=[]
target_arr=[]

#else mount the drive and give path of the parent-folder containing all category images folders.
datadir='./cs306_project2/dataset/ExtractedSet/dataset'  #path to the main data folder

Categories=['free', 'green', 'sheep', 'slow', 'stop', 'yellow']

for i in Categories:
  print(f'loading... category : {i}')
  path=os.path.join(datadir,i)    #working with file/directory paths, common pathname manipulations
  for img in os.listdir(path):
    img_array=imread(os.path.join(path,img))
    img_resized=resize(img_array,(224,224,3))
    flat_data_arr.append(img_resized.flatten())
    target_arr.append(Categories.index(i))
  print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
df



loading... category : free


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/yukyunglee/Documents/GitHub/cs306_project2/dataset/dataset_\\free'

In [8]:
picklePath = "../DataFormating/DataFrame/testdataframe.pkl"

#Load in the pickled dataframe created in FormatData.ipynb
with open(picklePath, 'rb') as f:
    df = pickle.load(f)

#Show the shape of the dataframe
print("Dataframe shape:", df.shape)

Dataframe shape: (2211, 150529)


In [9]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')

Splitted Successfully


In [11]:
# Model definition and application


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224, 224)),  #making 2D array to 1D so 28*28 = 784 will be the new batch size
    tf.keras.layers.Dense(128, activation='relu'),  #each neurons with their own weight will calculate through the 784 batch and also add their bias.
    tf.keras.layers.Dense(64, activation='relu'),                       #Final layer producing 10 outputs
    tf.keras.layers.Dense(6, activation='softmax')  #Final layer producing 6 outputs
])

print("Model Configured")

model.compile(optimizer='adam',   #Chooses Adam to update the model’s weights during training. Adam is a popular gradient descent method
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),   #Tells Keras how to measure “how wrong” the predictions are.
              metrics=['accuracy'])
model = make_pipeline(
    StandardScaler(),
    LogisticRegression(solver="lbfgs", max_iter=3000)
)
#(train_images, train_labels), (test_images, test_labels) = df.load_data()
#test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
model.fit(x_train, y_train)
predictions = model.predict_proba(x_test)
y_pred = model.predict(x_test)
print("Model Fitted")

acc_score = accuracy_score(y_test, y_pred)
print("Accuracy Score: " + str(acc_score*100) + "%")

C:\Users\makjo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model Configured
Model Fitted
Accuracy Score: 95.93679458239278%


In [12]:
import numpy as np
import matplotlib.pyplot as plt

IMG_SHAPE = (224, 224, 3)  

def to_label_indices(y):            #standardise labels into a 1D array of integer class indices (sklearn APIs expect 1D labels)
    y = np.asarray(y)
    return (np.argmax(y, axis=1) if y.ndim == 2 else y.astype(int, copy=False))

def to_images(X, img_shape):        #Get image tensors for visualisation
    X = np.asarray(X)
    if X.ndim == 4:   # already (N (Batch size),H (Height),W (Width),C (Channels))
        return X
    if X.ndim == 2:  
        return X.reshape((X.shape[0],) + img_shape)


test_labels = to_label_indices(y_test)  
test_images = to_images(x_test, IMG_SHAPE)   # for left panel image display



predictions = model.predict_proba(x_test)         
class_names = list(model.classes_)                # ensure consistent class order


def plot_image(i, pred_probs_i, labels, images, class_names=class_names):   #show the i-th image with a title summarising prediction vs truth (on left side)
    true_idx = int(labels[i])
    pred_idx = int(np.argmax(pred_probs_i))
    if images is not None:
        img = images[i]
        plt.imshow(img.astype(np.uint8) if img.max() > 1.0 else img)
    else:
        plt.text(0.5, 0.5, "No image available", ha='center', va='center')
    plt.axis('off')
    title = f"Pred: {class_names[pred_idx]} | True: {class_names[true_idx]}"
    plt.title(title, color=('blue' if pred_idx == true_idx else 'red'), fontsize=10)

def plot_value_array(i, pred_probs_i, labels, class_names=class_names):        #bar chart of the predicted class probabilities for sample i (on right side)
    true_idx = int(labels[i])
    pred_idx = int(np.argmax(pred_probs_i))
    bars = plt.bar(range(len(pred_probs_i)), pred_probs_i)
    bars[pred_idx].set_color('red')
    bars[true_idx].set_color('blue')
    plt.ylim([0, 1.0])
    plt.xticks(range(len(pred_probs_i)), class_names, rotation=45, ha='right')
    plt.ylabel("probability")


i = 120  # choose any valid index
plt.figure(figsize=(7,3))
plt.subplot(1,2,1); plot_image(i, predictions[i], test_labels, test_images, class_names)
plt.subplot(1,2,2); plot_value_array(i, predictions[i], test_labels, class_names)
plt.tight_layout(); plt.show()

y_pred_idx = np.argmax(predictions, axis=1)
acc = (y_pred_idx == test_labels).mean()
print(f"Test accuracy: {acc*100:.2f}%")
print("Class order:", class_names)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


x_Train, x_Test, y_Train, y_Test = train_test_split(x, y, test_size=0.2, random_state=77, stratify=y)

model = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=512, random_state=42)),   
    ("mlp", MLPClassifier(hidden_layer_sizes=(1024,128), activation='relu', alpha=1e-3, batch_size=64, learning_rate_init=5e-4, 
                          early_stopping=True,
                          n_iter_no_change=15,
                          max_iter=400,
                          random_state=42))
])

model.fit(x_Train, y_Train)
predictions = model.predict_proba(x_Test)
print("Accuracy:", accuracy_score(y_Test, predictions.argmax(1)))

# Reuse the plotting cell above; just set:
X_test, y_test = x_Test, y_Test

ValueError: invalid literal for int() with base 10: 'stop'

In [ ]:
#Export the trained model using pickle
import pickle
import os

# Use relative path in the models directory
modelPath = "yy_mlp_model.pkl"  # Saves in current directory (models/)

# OR use absolute path within your project
# modelPath = os.path.join(os.getcwd(), "nk_mlp_model.pkl")

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(modelPath) if os.path.dirname(modelPath) else '.', exist_ok=True)

try:
    with open(modelPath, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model Exported successfully to {os.path.abspath(modelPath)}")
except PermissionError:
    print(f"Permission denied: Unable to save to {modelPath}")
except Exception as e:
    print(f"An error occurred: {e}")

Model Exported successfully to c:\Users\makjo\GithubPrograms\cs306_project2\models\nk_mlp_model.pkl
